In [1]:
#when running from HDT: uncomment this
import sys, os
path_dir = os.getcwd()
sys.path.append(path_dir+"/Functions")
from general_functions import * 
from muscle_functions import * 
from constants_functions import * 
from plot_functions import *
from save_functions import *
import xarray as xr

#must include 'longmuscle' for long muscle parameters, and 'shortmuscle' for short muscle parameters
experiment_name = 'full_curve_longmuscle'#'zoomed'

#model type determines whether Huxley or Hill functions are used, v_type determines the used vce equation in the Huxley model (old is used) and FL_.. determines the used FL model (hill is used)
model_type,v_type,FL_type='huxley','v_old','FL_hill' #'huxley','hill'---'v_old','v_new'---'FL_0','FL_hill','FL_huxley'
FV_type='FV_hill'
hill_type = 'thijs'         #thijs, stroeve #determines the hill model used. 'thijs' for hill_expanded, 'stroeve' for hill_simple
a_func=True                 #use an activation function

save_name = '2newstraightcurvenewpos_' #determines the save name
specific_save_name = 'stiff_start_newpos'

#experiment parameters
fs=2500                     #steps/second
muscle_number = 1           #nr of muscles used    
tot_bridge_nr = 10000       #number of cross-bridges in the Huxley Model
l_mtc0 = 0.32               #muscle length
mus_vals = muscle_vals_func()
h_step = 1/fs               #stepsize         (s)
fmax=1                      #determines the fmax constant
t0 = 0.0                    #time start point (s)
tf = 82.0                   #time end point   (s)
step_nr = int(fs*(tf-t0))   #total steps



#iterate and produce FL curve data for all these velocities
velocity_scale_factor_list = np.array([ 0.1,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
        2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
        4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
        6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,
        9.  ,  9.25,  9.5 ,  9.75, 10.  ])#np.array([0.1,0.2,0.5,1.0,1.5,2.0,3.0,5.0,6.5,8.5,10.0])#

for scale_list_nr in range(len(velocity_scale_factor_list)):
    v_mtc = np.zeros((step_nr+1))
    velocity_scale_factor = velocity_scale_factor_list[scale_list_nr]

    velocity =0.8*mus_vals[0][8]*velocity_scale_factor#0.35*mus_vals[0][8]

    t1_start = 1.1#start velocity input
    t1_end = t1_start+2.25/velocity_scale_factor#1.75/velocity_scale_factor #end   velocity input
    v_mtc[int(t1_start*fs+1):int(t1_end*fs)]=velocity#add negative stretch at low velocity
    

    t3_start = 24.01#start velocity input
    t3_end = t3_start+3.79/velocity_scale_factor #end   velocity input
    v_mtc[int(t3_start*fs+1):int(t3_end*fs)]=-velocity
    t4_start = 65.41#start velocity input
    t4_end = t4_start+1.54/velocity_scale_factor #end   velocity input
    v_mtc[int(t4_start*fs+1):int(t4_end*fs)]=velocity
    time_vector=np.linspace(t0,tf,step_nr+1)    
    l_mtc = l_mtc0*np.ones(step_nr+1)+h_step*np.cumsum(v_mtc)

    K_SE= np.zeros(step_nr+1)
    K_CE= np.zeros(step_nr+1)
    #plt.plot(time_vector,l_mtc)





    hill_type = 'thijs'
    model_type='huxley'
    framemax_variable=-1#500
    for ii in range(5):#--------
        a=0
        #hill_type = 'thijs'#--------
        #model_type='hill'#--------
        if ii == 4:
            model_type='huxley'
            v_type='v_old'
            FV_type='FV_hill'
            model_name='hux_vold_'
        if ii == 11:
            model_type='huxley'
            v_type='v_new'
            FV_type='FV_hill'
            model_name='hux_vnew_'
        if ii == 0:
            model_type='hill'
            hill_type='thijs'
            FV_type='FV_hill'
            model_name='hill_thijs_fvhill_'
        if ii == 1:
            model_type='hill'
            hill_type='thijs'
            FV_type='FV_0'
            model_name='hill_thijs_fv0_'
        if ii == 2:
            model_type='hill'
            hill_type='stroeve'
            FV_type='FV_hill'
            model_name='hill_stroeve_fvhill_'
        if ii == 3:
            model_type='hill'
            hill_type='stroeve'
            FV_type='FV_0'
            model_name='hill_stroeve_fv0_'    
        model_save_name = model_name+str(velocity_scale_factor)
        if model_type=='huxley':
            states_hux = np.zeros((step_nr+1,tot_bridge_nr*2+2))
            #mus_vals_hux=mus_vals[0]
            mus_vals_hux = muscle_vals_func()
            mus_vals_changed,states_hux = constant_altering_function(mus_vals_hux[0],l_mtc0,states_hux,muscle_number,tot_bridge_nr,model_type,hill_type,fmax,experiment_name,specific_save_name)
            STIM = np.ones(len(time_vector))
            STIM[0:int(0.5*fs)]=0
            ti = t0
            calculated_muscle_forces = np.zeros((step_nr+1,3))

           # specific_save_name = 'hux'+str(velocity_scale_factor)
            while a < step_nr:
                    t_counter = ti
                    ti += h_step

                    states_previous_step = states_hux[a]
                    states_previous_step2 = states_hux[a-1]
                    calculated_muscle_forces_previous_step = calculated_muscle_forces[a]
                    F_per_step, state_current_step,K_SE_single,K_CE_single = huxley(l_mtc[a],v_mtc[a],states_previous_step,t_counter,h_step,mus_vals_changed,tot_bridge_nr,STIM[a],False,True,FL_type,v_type,states_previous_step2,calculated_muscle_forces_previous_step)#calculates the muscle force per muscle per cross-bridge for the current length and velocity values 
                    a += 1
                    states_hux[a] = state_current_step
                    calculated_muscle_forces[a] = F_per_step
                    K_SE[a] = K_SE_single
                    K_CE[a] = K_CE_single
            lCE = states_hux[:,-2] 
            lSE = l_mtc - lCE      
            vCE = np.gradient(lCE,h_step)      
            vSE = np.gradient(lSE,h_step)     
            vCEhux = vCE
        if model_type=='hill':
            states2 = np.zeros((step_nr+1,2))
            mus_vals_hill = muscle_vals_func()
            mus_vals_changed2,states2 = constant_altering_function(mus_vals_hill[0],l_mtc0,states2,muscle_number,tot_bridge_nr,model_type,hill_type,fmax,experiment_name,specific_save_name)
            #print(mus_vals_changed2)
            ti = t0
            STIM = np.ones(len(time_vector))
            STIM[0:int(0.5*fs)]=0
            force_vector = np.zeros((step_nr+1,9))
            force_vector[0][6],force_vector[0][4]=1,0.2#FLFV
            force_vector[0][2]=1/8*mus_vals_changed2[7]
            force_vector[0][1]=1/4*mus_vals_changed2[7]
            calculated_hill_forces = np.zeros((step_nr+1))
            while a < step_nr:
                    t_counter = ti
                    ti += h_step

                    force_vector_previous_step = force_vector[a]    
                    states_previous_step = states2[a]
                    #force_vector_previous_stepl_mtc,v_mtc,states_previous_step,force_vector_previous_step,t_counter,h_step, mus_vals,STIM,FV_type='FV_hill',activation_function=True
                    F_hill, new_states = hill(l_mtc[a],v_mtc[a],states_previous_step,force_vector_previous_step,t_counter,h_step, mus_vals_changed2,STIM[a],hill_type,FV_type,a_func)#calculates the muscle force per muscle per cross-bridge for the current length and velocity values 
                    a += 1
                    states2[a] = new_states
                    calculated_hill_forces[a] = F_hill[0]
                    force_vector[a] = F_hill
            #lCE = l_mtc - 0.02
            lCE = states2[:,0]
            vCE = np.gradient(lCE,h_step)     
        if model_type=='huxley':
            K_SE2 = np.zeros(len(lSE))
            K_PE2 = np.zeros(len(lSE))
            for i in range(len(lSE)):
                K_SE2[i],K_PE2[i] = EE_Thelen(mus_vals_changed,lSE[i],0)
            #plot_huxley_full(time_vector, calculated_muscle_forces,states_hux,t0,tf,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals,lSE,lCE,vSE,vCE,l_mtc,v_mtc,K_SE2)
        if model_type=='huuuxley':
            plt.plot(l_mtc[1:],calculated_muscle_forces[:,0][1:])
        if model_type=='hiiill':
            plt.plot(l_mtc[1:],calculated_hill_forces[1:])

        if model_type =='huxley':
            save_huxley(l_mtc,tot_bridge_nr,states_hux,STIM,calculated_muscle_forces,v_mtc,lCE,lSE,K_SE2,time_vector,save_name,model_save_name)
        if model_type =='hill':
            save_hill(l_mtc,calculated_hill_forces,force_vector,v_mtc,STIM,lCE,vCE,time_vector,save_name,model_save_name)
        model_type='hill'
        if ii == 1:
            FV_type='FV_0'
        if ii == 2:
            FV_type='FV_hill'  
            hill_type = 'stroeve'

/mnt/Data/axel/2.3_sm_improve/Functions/muscle_functions.py:401: RuntimeWarning: divide by zero encountered in double_scalars
  f_rel = F_CE_prev/( activation*Fmax*FL_prev)#F_SE_prev/(Fmax*activation*FL_prev)
/mnt/Data/axel/2.3_sm_improve/Functions/muscle_functions.py:401: RuntimeWarning: invalid value encountered in double_scalars
  f_rel = F_CE_prev/( activation*Fmax*FL_prev)#F_SE_prev/(Fmax*activation*FL_prev)


<xarray.Dataset>
Dimensions:  (time: 205001)
Coordinates:
  * time     (time) float64 0.0 0.0004 0.0008 0.0012 ... 82.0 82.0 82.0 82.0
Data variables: (12/14)
    F_mtc    (time) float64 0.0 -0.0007579 -0.0007579 ... 0.9979 0.9979 0.9979
    v_mtc    (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    l_mtc    (time) float64 0.32 0.32 0.32 0.32 0.32 ... 0.32 0.32 0.32 0.32
    STIM     (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    lCE      (time) float64 0.2634 0.2634 0.2634 0.2634 ... 0.2628 0.2628 0.2628
    vCE      (time) float64 0.0 0.0 0.0 0.0 ... 2.22e-12 2.22e-12 2.22e-12
    ...       ...
    F_PE     (time) float64 0.0 -0.0007579 -0.0007579 ... -0.0008646 -0.0008646
    FL       (time) float64 0.2 0.999 0.999 0.999 ... 0.9987 0.9987 0.9987
    vmax     (time) float64 0.0 0.8015 0.8015 0.8015 ... 1.602 1.602 1.602 1.602
    FV       (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    vce      (time) float64 0.0 0.0